# La recherche textuelle

## Algorithme de Boyer-Moore de recherche textuelle

In [15]:
# texte
motif="AT-THAT"
texte="WHICH-FINALLY-HALTS.--AT-THAT-POINT.-MAY-BE-NOT-IF-IT-IS-AT-THAT-POINT."
alphabet="WHIC-FNALYTS.OBE"

### Retour sur la recherche simple

Le script ci-dessous permet d'effectuer une recherche simple. i pointe un caractère du texte. j pointe un caractère du motif. Après chaque décalage du motif d'une case vers la droite, i et j sont initialisés à la fin du motif pour j et sur la lettre du texte coincidant avec la dernière lettre du motif pour i. La comparaison se fait à partir de la fin du motif et en remontant la chaîne.

In [16]:
def recherche(motif,texte):
    
    n=len(texte)
    m=len(motif)
    p=[]
    
    i=m-1
    while i<n:
        j=m-1
        while j>-1 and texte[i]==motif[j]:
            j=j-1 
            i=i-1
            
        if j==-1:
            p.append(i+1)
            i=i+(m-j)
        else:
            i=i+(m-j) # m-j permet d'avancer le motif de 1 case vers la droite
    return p

### Algorithme de boyer moore limité à la règle du mauvais caractère

L'algorithme de boyer moore parcours les lettres de la même manière que notre algorithme de recherche précédent. La seule différence est que quand deux lettres ne correspondent pas; le décalage est optimisé ie il est plus grand que celui de la recherche simple.

Le décalage est pré-calculé (valeurs dans le dictionnaire d1 dont les clés sont les lettres de l'alphabet) à partir du motif selon la règle du mauvais caractère.

### Présentation de la règle du mauvais caractère

```
      i
WHICH-FINALLY-HALTS.--AT-THAT-POINT
AT-THAT
      j

(1) F et T ne correspondent pas. Or dans le motif, il n'y a pas de F. Il est donc inutile de faire correspondre ce F avec une lettre du motif. Le motif peut donc être décalé aprés le F.

             i
WHICH-FINALLY-HALTS.--AT-THAT-POINT
       AT-THAT
             j
             
(2) - et T ne correspondent pas. Or dans le motif, il y a un -. On peut donc décaler le motif de manière à faire correspondre ce caractère avec le texte (si il y a plusieurs -, on se limite par choix seulement à celui qui est le plus à droite et seulement s'il permet un décalage vers la droite).

                 i
WHICH-FINALLY-HALTS.--AT-THAT-POINT
           AT-THAT
                 j
        
                i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
           AT-THAT
                j

Le T correspond mais pas le L. Pour la même remarque (1), on décale le motif aprés le L.

                       i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
                 AT-THAT
                       j


                      i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
                 AT-THAT
                      j

                     i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
                 AT-THAT
                     j
                     
Le - ne correspond pas au H. Pour la même remarque (2), on aligne les deux -.

                         i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
                   AT-THAT
                         j


                        i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
                   AT-THAT
                        j

Le - ne correspond pas au A. (2) on aligne les deux -.


                            i        
WHICH-FINALLY-HALTS.--AT-THAT-POINT
                      AT-THAT
                            j
                            
Le mot sera bien trouvé.

```

### Construction de la table de décalage du mauvais caractère

Exemple avec AT-THAT

Le pré-traitement consiste à calculer un dictionnnaire de décalage pour i si une lettre de correspond pas. Les clés de ce dictionnaire sont les lettres de l'alphabet utilisé.

Si la lettre n'est pas dans le motif, le décalage est alors égal à la longueur du motif.

Si la lettre est dans le motif, le décalage est alors égal à sa position comptée à partir de la fin. Si plusieurs lettres sont iddentiques, la position la plus à droite (sauf la dernière position) de chaque lettre est retenue.

```
Exemple:

A T - T H A T      longueur du motif 7
6 5 4 3 2 1 0

dictionnaire des décalages pour chaque caractère

A     1
T     3 (T en position 0 est inutile)
-     4
H     2
Autre 7

```

Attention on peut avoir des décalages dans le mauvais sens ou alors un décalage nul ! Pour cette raison, on doit comparer le décalage produit avec celui de la recherche simple

```
Situation 1

           i
.....XXXXXX-.....
     AT-THAT
     
selon la table i sera incrémenté de 4

               i
.....XXXXXX-.....
         AT-THAT
         
Situation 2

          i
.....XXXXX-T.....
     AT-THAT

Selon la table i sera incrémenté de 4

              i
.....XXXXX-T.....
        AT-THAT
        
Situation 3

        i
.....XXXAHAT.....
     AT-THAT
     
Selon la table i sera incrémenté de 1

         i
.....XXXAHAT.....
   AT-THAT
   
Les A sont bien alignés mais le motif a reculé ! On préfére alors augmenter i de m-j comme dans le recherche simple. D'où la fonction max dans le script
```

In [10]:
# traitement pour la règle du mauvais caractère MC
def pretraitement_MC(motif,alphabet):
    m=len(motif)
    d1=dict()
    for c in alphabet:
        d1[c]=m
    for i in range(0,m-1):
        d1[motif[i]]=m-1-i
    return d1

In [11]:
assert pretraitement_MC("AT-THAT","WHIC-FNALYTS.OBE")=={'W': 7, 'H': 2, 'I': 7, 'C': 7, '-': 4, 'F': 7, 'N': 7, 'A': 1,
                                                        'L': 7, 'Y': 7, 'T': 3, 'S': 7, '.': 7, 'O': 7, 'B': 7,'E': 7}

In [12]:
print(motif)
print(alphabet)
pretraitement_MC(motif,alphabet)

AT-THAT
WHIC-FNALYTS.OBE


{'W': 7,
 'H': 2,
 'I': 7,
 'C': 7,
 '-': 4,
 'F': 7,
 'N': 7,
 'A': 1,
 'L': 7,
 'Y': 7,
 'T': 3,
 'S': 7,
 '.': 7,
 'O': 7,
 'B': 7,
 'E': 7}

In [ ]:
# recherche avec la règle du mauvais caractère
def recherche(motif,texte,alphabet):
    
    d=pretraitement_MC(motif,alphabet)
    n=len(texte)
    m=len(motif)
    
    i=m-1
    while i<n:
        print(texte)
        print(" "*(i-m+1)+motif)
        print()
        j=m-1
        while j>-1 and texte[i]==motif[j]:
            j=j-1
            i=i-1
            
        if j==-1:
            return i+1,True
        else:
            i=i+max(d[texte[i]],m-j) # le décalage m-j est celui de la recherche simple dans le pire des cas
                                     # en effet, si i a trop baissé d[texte[i]] ne permet pas de remonter suffisament haut
    return i+1,False

In [ ]:
print(motif)
print(texte)
recherche(motif,texte,alphabet)

### A retenir

1. L'Algorithme de boyer-moore est un algorithme de recherche textuelle
2. Basé sur un pré traitement du motif avec un calcul de tables de décalage
3. Il compare le texte et le motif en partant par la fin du motif. Si les lettres ne correspondent pas; il décale le motif
4. Amélioration de type 5 fois mieux qu'un algo de recherche de base
5. Pas terrible si peu de lettre dans l'alphabet
